In [5]:
from glob import glob
import numpy as np
import open3d as o3d
import os
import tifffile
from PIL import Image

out_path = "face_normals_subdivision"
viz_path = f'{out_path}/viz'
os.makedirs(viz_path, exist_ok=True)
os.makedirs(out_path, exist_ok=True)

In [ ]:
w, h = 8096, 7888
# meshes = glob('../../scroll1_gp_meshes/*.obj')
meshes = glob('../../scroll1_gp_meshes/*.obj')

all_coords = []
all_norms = []

for mesh_path in meshes:
    mesh = o3d.io.read_triangle_mesh(mesh_path)
    subdivided_mesh = mesh.subdivide_midpoint(number_of_iterations=4)
    vertices = np.asarray(subdivided_mesh.vertices, dtype=np.float32)
    triangles = np.asarray(subdivided_mesh.triangles)

    v0 = vertices[triangles[:, 0]]
    v1 = vertices[triangles[:, 1]]
    v2 = vertices[triangles[:, 2]]

    face_normals = np.cross(v1 - v0, v2 - v0)
    face_normals = face_normals / np.linalg.norm(face_normals, axis=1, keepdims=True)
    face_normals = face_normals.astype(np.float32)
    face_centers = ((v0 + v1 + v2) / 3).astype(np.float32)

    all_coords.append(face_centers)
    all_norms.append(face_normals)

# Combine all points and normals
coords = np.vstack(all_coords)
norms = np.vstack(all_norms)

In [3]:
rounded_z = np.round(coords[:,2]).astype(int)
req_slices = np.unique(rounded_z)
print(f"Will process slices from {req_slices.min()} to {req_slices.max()}")
print(f"Total number of slices: {len(req_slices)}")

Will process slices from 10004 to 10998
Total number of slices: 995


In [4]:

for i in range(0, len(meshes), 2):
    batch_meshes = meshes[i:i+2]
    print(f"Processing batch {i//2 + 1} of {(len(meshes) + 1)//2}")

    all_coords = []
    all_norms = []

    for mesh_path in batch_meshes:
        print(f"Processing mesh: {os.path.basename(mesh_path)}")
        mesh = o3d.io.read_triangle_mesh(mesh_path)
        subdivided_mesh = mesh.subdivide_midpoint(number_of_iterations=4)
        vertices = np.asarray(subdivided_mesh.vertices, dtype=np.float32)
        triangles = np.asarray(subdivided_mesh.triangles)

        v0 = vertices[triangles[:, 0]]
        v1 = vertices[triangles[:, 1]]
        v2 = vertices[triangles[:, 2]]

        face_normals = np.cross(v1 - v0, v2 - v0)
        face_normals = face_normals / np.linalg.norm(face_normals, axis=1, keepdims=True)
        face_normals = face_normals.astype(np.float32)
        face_centers = ((v0 + v1 + v2) / 3).astype(np.float32)

        all_coords.append(face_centers)
        all_norms.append(face_normals)

        del subdivided_mesh, vertices, triangles

    # Combine points and normals for this batch
    coords = np.vstack(all_coords)
    norms = np.vstack(all_norms)

    rounded_z = np.round(coords[:,2]).astype(int)
    req_slices = np.unique(rounded_z)

    print(f"Processing slices from {req_slices.min()} to {req_slices.max()}")
    print(f"Total slices in batch: {len(req_slices)}")

    exp_factor = 2.0

    for zslice in req_slices:
        # Load existing images if they exist
        if os.path.exists(f'{out_path}/{zslice}.tif'):
            img = tifffile.imread(f'{out_path}/{zslice}.tif')
            viz_img = np.array(Image.open(f'{viz_path}/{zslice}.jpg'))
        else:
            img = np.zeros((h, w, 3), dtype=np.uint16)
            viz_img = np.zeros((h, w, 3), dtype=np.uint8)

        mask = rounded_z == zslice
        slice_points = coords[mask]
        normals = norms[mask]
        print(f"Processing slice {zslice} with {len(slice_points)} points")

        if len(slice_points) > 0:
            for i in range(len(slice_points)):
                x = int(round(slice_points[i, 0]))
                y = int(round(slice_points[i, 1]))

                normal = normals[i] / np.linalg.norm(normals[i])
                normal_rgb = ((normal + 1) * 32767.5).astype(np.uint16)
                normal_viz = (np.abs(normal) * 255).astype(np.uint8)

                num_steps = int(2 * exp_factor + 1)
                for t in np.linspace(-exp_factor, exp_factor, num_steps):
                    new_x = int(round(x + t * normal[0]))
                    new_y = int(round(y + t * normal[1]))
                    if 0 <= new_x < w and 0 <= new_y < h:
                        img[new_y, new_x] = normal_rgb
                        viz_img[new_y, new_x] = normal_viz

            tifffile.imwrite(f'{out_path}/{zslice}.tif', img, compression='zlib')
            Image.fromarray(viz_img).save(f'{viz_path}/{zslice}.jpg', quality=90)

    # Clear batch data
    del coords, norms, all_coords, all_norms

Processing slice 10004 with 112 points
Processing slice 10005 with 1296 points
Processing slice 10006 with 8837 points
Processing slice 10007 with 36642 points
Processing slice 10008 with 94317 points
Processing slice 10009 with 140839 points
Processing slice 10010 with 157063 points
Processing slice 10011 with 159125 points
Processing slice 10012 with 159488 points
Processing slice 10013 with 158559 points
Processing slice 10014 with 158522 points
Processing slice 10015 with 157024 points
Processing slice 10016 with 156947 points
Processing slice 10017 with 155372 points
Processing slice 10018 with 155347 points
Processing slice 10019 with 153921 points
Processing slice 10020 with 153825 points
Processing slice 10021 with 152949 points
Processing slice 10022 with 153545 points
Processing slice 10023 with 153162 points
Processing slice 10024 with 154998 points
Processing slice 10025 with 155048 points
Processing slice 10026 with 157469 points
Processing slice 10027 with 157358 points
P

KeyboardInterrupt: 

In [6]:
import numpy as np
import tifffile as tiff

def calculate_nonzero_percentage(tiff_path):
    """
    Calculate the percentage of nonzero voxels in a multipage TIFF file.

    Parameters:
        tiff_path (str): Path to the multipage TIFF file.

    Returns:
        float: Percentage of nonzero voxels in the volume.
    """
    # Load the multipage TIFF file
    with tiff.TiffFile(tiff_path) as tif:
        volume = tif.asarray()

    # Ensure the data is loaded as a numpy array
    if not isinstance(volume, np.ndarray):
        raise ValueError("Failed to load the TIFF file as a numpy array.")

    # Count nonzero voxels
    nonzero_count = np.count_nonzero(volume)

    # Calculate total number of voxels
    total_voxels = volume.size

    # Calculate percentage
    percentage = (nonzero_count / total_voxels) * 100

    return percentage

if __name__ == "__main__":
    # Path to your multipage TIFF file
    tiff_path = "/mnt/raid_nvme/datasets/nnUNet_raw/Dataset055_ScaledImages/labelsTr/chunk05_cell_yxz_009_006_021.tif"

    try:
        percentage = calculate_nonzero_percentage(tiff_path)
        print(f"Percentage of nonzero voxels in the volume: {percentage:.2f}%")
    except Exception as e:
        print(f"An error occurred: {e}")


Percentage of nonzero voxels in the volume: 11.85%


In [10]:
import zarr
import tifffile

z = zarr.open('/mnt/raid_nvme/s1.zarr', mode='r')
zs = z[5000]
tifffile.imwrite('/home/sean/Desktop/raws1.tif',zs)